In [12]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os

In [13]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
# Create a Pinecone index
pc.create_index(
    name="rag",
    dimension=1536, #the number of dimension that the openAI embedding have
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '98f5f52a772f83bb55abac705f79d49c', 'Date': 'Fri, 23 Aug 2024 13:02:46 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [14]:
# Load the review data
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily White',
  'review': 'Engaging and clear explanations. Makes complex topics understandable.',
  'subject': 'Physics',
  'stars': 5},
 {'professor': 'Prof. Robert Green',
  'review': 'Very strict grading, but knowledgeable. Be prepared to work hard.',
  'subject': 'Chemistry',
  'stars': 4},
 {'professor': 'Dr. Lisa Turner',
  'review': 'Monotone lectures. Hard to stay focused in class.',
  'subject': 'History',
  'stars': 2},
 {'professor': 'Prof. Michael Davis',
  'review': 'Passionate about the subject, but assignments are vague.',
  'subject': 'English Literature',
  'stars': 3},
 {'professor': 'Dr. Sarah Johnson',
  'review': 'Incredibly helpful during office hours. Really cares about students.',
  'subject': 'Biology',
  'stars': 5},
 {'professor': 'Prof. James Brown',
  'review': 'Too fast-paced, but comprehensive. You learn a lot if you can keep up.',
  'subject': 'Calculus',
  'stars': 4},
 {'professor': 'Dr. Rachel Miller',
  'review': 'Always prepared

In [15]:
# Load the review data
from openai import OpenAI
data = json.load(open("reviews.json"))

processed_data = []
client = OpenAI()


In [16]:
# Create embeddings for each review
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )

In [17]:
# Create embeddings for each review
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )

In [18]:
processed_data[0]

{'values': [-0.018490363,
  0.020048616,
  -0.018949492,
  0.013307774,
  0.003403465,
  -0.0003828246,
  -0.014386031,
  0.03561725,
  0.010274743,
  0.032194655,
  0.065836266,
  0.02825728,
  -0.03425378,
  -0.031777266,
  0.034225952,
  -0.0052869357,
  -0.020939048,
  0.022191217,
  0.029105972,
  0.031944223,
  0.037592895,
  -0.019853836,
  0.023777299,
  0.06054933,
  -0.033446826,
  -0.057265863,
  -0.0054678046,
  0.00511998,
  -0.019492097,
  -0.0011156478,
  0.04109897,
  -0.025502509,
  -0.01782254,
  -0.050281543,
  -0.0020608616,
  0.027269458,
  -0.004559982,
  -0.014733856,
  -0.0027495543,
  0.030747706,
  0.04683112,
  0.011352999,
  -0.045551125,
  -0.018587753,
  0.032890305,
  0.022511216,
  -0.027951196,
  -0.025196424,
  0.037453767,
  0.02835467,
  -0.015610374,
  0.016598197,
  0.022914693,
  0.006017368,
  -0.053091966,
  0.035589427,
  0.011471259,
  0.034615517,
  -0.050615456,
  -0.020577312,
  0.03661899,
  -0.0036660726,
  0.014845159,
  0.018977318,
  -

In [19]:
# Insert the embeddings into the Pinecone index
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")


Upserted count: 62


In [20]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 31}},
 'total_vector_count': 31}